# Table of Contents
1  Image概述
2  Image operator(mask, clip, select, addBands, export)
3  ImageCollection概述
4  ImageCollection operator(filter, select, set/get, calculation, toList)
5  Example 1. Calculate NDVI for Jiangxi and Extract NDWI for poyang Lake between 1982 to 2021;
6  Example 2. Download huge ImageCollection from google earth engine; -> Download Images
7  Example 3. Extract NDVI values of climate stations from 1982 to 2021; -> Spatial analysis
8  Example 4. Supervised Classification -> Images Classification
9  Example 5. Unsupervised Classification (clustering) 

In [1]:
import ee, os
import geemap
# ee.Initialize()
Map = geemap.Map()
Map = geemap.Map(center=(39, 110), zoom=4)
Map.add_basemap('HYBRID')

In [2]:
Map

Map(center=[39, 110], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [2]:

# local shape file 
f_shp = r'D:\08.专业数据\A1_database\A2_images\00.基础地理数据\stations_2k_dd\stations_2k_gcs.shp' #添加面shp（文件名不能有“_”和中文）

# convert local shape file to ee.feature
shp_ee = geemap.shp_to_ee(f_shp)

# add ee.feature to the map
# Map.addLayer(eeObject, visParams, name, shown, opacity)
# Map.addLayer(shp_ee, {}, 'Climate stations')

In [3]:
# https://developers.google.com/earth-engine/datasets/catalog
# Climate datasets
# =======================================
# ERA  https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_MONTHLY
# CRU  https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
# NECP
# CFSR
# TRMM
# =======================================
year = 2010
col_era5 = ee.ImageCollection("ECMWF/ERA5/MONTHLY").filterDate(str(year), str(year+1))
col_cru3 = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").filterDate(str(year), str(year+1))


In [5]:
col_era5.first().propertyNames().getInfo()

['system:time_start',
 'month',
 'year',
 'system:footprint',
 'system:time_end',
 'system:version',
 'system:id',
 'system:asset_size',
 'system:index',
 'system:bands',
 'system:band_names']

In [12]:
col_era5.first().get('system:band_names').getInfo()

['mean_2m_air_temperature',
 'minimum_2m_air_temperature',
 'maximum_2m_air_temperature',
 'dewpoint_2m_temperature',
 'total_precipitation',
 'surface_pressure',
 'mean_sea_level_pressure',
 'u_component_of_wind_10m',
 'v_component_of_wind_10m']

In [11]:
# tavg_era5 = col_era5.select('mean_2m_air_temperature').mean()
# prcp_era5 = col_era5.select('total_precipitation').sum()


In [18]:
col_era5.select('mean_2m_air_temperature').aggregate_array('system:index').getInfo()

['201001',
 '201002',
 '201003',
 '201004',
 '201005',
 '201006',
 '201007',
 '201008',
 '201009',
 '201010',
 '201011',
 '201012']

In [26]:
# extract image values by points 
# Convert ImageCollection to single Image by toBands()
outFile1 = r'D:\100.Geo_Worker\era5_{}.csv'.format(year)
outFile2 = r'D:\100.Geo_Worker\cru3_{}.csv'.format(year)
if os.path.exists(outFile1):
    os.remove(outFile1)
# geemap.extract_values_to_points()
image_tavg_era5 = col_era5.select('mean_2m_air_temperature').toBands()
geemap.extract_values_to_points(shp_ee, image_tavg_era5, outFile1)

# Usage: from ee
# ee.Image().sampleRegions(collection, properties, scale, projection, tileScale, geometries)
# https://github.com/monarch-beluga/Study-notes/blob/master/python/jupyter/geemap/work/%E6%8C%89%E7%82%B9%E6%8F%90%E5%8F%96.md


Generating URL ...
Please wait ...
Data downloaded to D:\100.Geo_Worker\era5_2010.csv


In [4]:
fscale = col_era5.first().projection().nominalScale()

In [5]:
image_tavg_era5 = col_era5.select('mean_2m_air_temperature').toBands()
tavg_exm = image_tavg_era5.reduceRegions(shp_ee, ee.Reducer.mean(), scale=fscale) 

In [20]:
tavg_exm.first().propertyNames().getInfo()

['201001_mean_2m_air_temperature',
 '201002_mean_2m_air_temperature',
 '201003_mean_2m_air_temperature',
 '201004_mean_2m_air_temperature',
 '201005_mean_2m_air_temperature',
 '201006_mean_2m_air_temperature',
 '201007_mean_2m_air_temperature',
 '201008_mean_2m_air_temperature',
 '201009_mean_2m_air_temperature',
 '201010_mean_2m_air_temperature',
 '201011_mean_2m_air_temperature',
 '201012_mean_2m_air_temperature',
 'system:index',
 'ELV',
 'LAT',
 'LON',
 'SID',
 'x',
 'y']

In [23]:
tavg_exm.first().get('SID').getInfo()

'AFM00040948'

In [22]:
tavg_exm.first().get('201001_mean_2m_air_temperature').getInfo()

268.54058837890625

In [25]:
tavg_exm.aggregate_array('201001_mean_2m_air_temperature').getInfo()

[268.54058837890625,
 299.069091796875,
 292.48870849609375,
 300.03814697265625,
 300.26116943359375,
 300.1159362792969,
 300.2377624511719,
 299.63433837890625,
 299.5244445800781,
 299.89739990234375,
 288.113037109375,
 300.04913330078125,
 293.4880676269531,
 297.85693359375,
 293.57794189453125,
 287.4007568359375,
 287.54534912109375,
 285.8267517089844,
 300.0772399902344,
 286.21539306640625,
 284.47662353515625,
 299.6110534667969,
 277.0090026855469,
 280.4745178222656,
 279.59674072265625,
 282.2865295410156,
 270.4917907714844,
 268.8697204589844,
 271.8254089355469,
 268.59417724609375,
 271.005615234375,
 267.1066589355469,
 270.759521484375,
 270.27508544921875,
 271.08001708984375,
 268.7650451660156,
 265.81683349609375,
 270.1841735839844,
 271.5657043457031,
 272.7023010253906,
 270.5186767578125,
 269.796630859375,
 270.0454406738281,
 271.32611083984375,
 267.7849426269531,
 269.63812255859375,
 267.9547119140625,
 271.8510437011719,
 272.6667785644531,
 272.1169

In [7]:
df = geemap.ee_to_df(tavg_exm)

In [10]:
df.head(10)

,201001_mean_2m_air_temperature,201002_mean_2m_air_temperature,201003_mean_2m_air_temperature,201004_mean_2m_air_temperature,201005_mean_2m_air_temperature,201006_mean_2m_air_temperature,201007_mean_2m_air_temperature,201008_mean_2m_air_temperature,201009_mean_2m_air_temperature,201010_mean_2m_air_temperature,201011_mean_2m_air_temperature,201012_mean_2m_air_temperature,ELV,LAT,LON,SID,x,y
0,268.540588,266.812073,276.442413,283.107208,286.302521,289.113556,293.154022,292.641479,289.014160,284.440094,278.059845,266.614197,1791.3,34.5660,69.2120,AFM00040948,-3.574636e+06,4.430595e+06
1,299.069092,299.972656,301.380981,301.216980,301.420502,301.298431,301.319702,301.364258,300.855774,300.856476,299.502106,298.578033,20.0,6.1170,81.1330,CE000043497,-3.449187e+06,1.109966e+06
2,292.488708,293.359863,294.675079,294.752472,295.062195,294.384216,293.650879,293.476410,293.393127,293.355164,292.935760,292.153198,1880.0,6.9670,80.7670,CE000434730,-3.466669e+06,1.205151e+06
3,300.038147,300.049286,300.224426,300.476959,300.773407,300.463226,300.136139,299.939636,299.891235,299.928558,300.170532,299.689148,9.1,6.8456,158.3064,FMC00914482,5.591289e+06,2.066475e+06
4,300.261169,299.707947,300.211731,300.555786,301.220520,300.956848,300.696411,300.395233,300.670258,300.470520,300.625244,300.549927,13.4,9.4833,138.0833,FMW00040308,3.262162e+06,1.413143e+06
5,300.115936,300.117340,300.294525,300.550171,300.847839,300.533844,300.186188,299.985870,299.958191,299.946991,300.249817,299.755646,36.6,6.9667,158.2167,FMW00040504,5.575997e+06,2.072301e+06
6,300.237762,300.066956,300.275787,300.677917,301.014496,300.555481,300.291138,300.119995,300.193146,300.114868,300.363556,300.247742,1.5,7.4500,151.8333,FMW00040505,4.865603e+06,1.780723e+06
7,299.634338,299.342926,299.564148,300.015106,300.784698,300.996979,300.761139,300.424408,300.370789,300.484802,300.699768,300.553864,85.3,13.3114,144.7364,GQC00914278,3.869497e+06,2.031908e+06
8,299.524445,299.242126,299.451721,299.904480,300.693085,300.891479,300.688782,300.348022,300.284729,300.382996,300.630249,300.454468,77.4,13.4836,144.7961,GQW00041415,3.869637e+06,2.051295e+06
9,299.897400,300.388519,300.583801,300.973450,301.355011,300.769531,300.274536,300.272614,300.299957,300.404724,299.985901,299.777527,30.0,-0.4790,104.5790,IDM00096179,-6.921613e+05,-2.733693e+04


In [21]:
# extract image values by points 
# Convert ImageCollection to single Image by toBands()
outFile1 = r'D:\100.Geo_Worker\era5_{}.csv'.format(year)
outFile2 = r'D:\100.Geo_Worker\cru3_max_{}.csv'.format(year)
outFile3 = r'D:\100.Geo_Worker\cru3_min_{}.csv'.format(year)
if os.path.exists(outFile2):
    os.remove(outFile2)
# geemap.extract_values_to_points()
image_tmax_cru3 = col_cru3.select('tmmx').toBands()
image_tmin_cru3 = col_cru3.select('tmmn').toBands()

geemap.extract_values_to_points(shp_ee, image_tmax_cru3, outFile2)
geemap.extract_values_to_points(shp_ee, image_tmin_cru3, outFile3)

Generating URL ...
Please wait ...
Data downloaded to D:\100.Geo_Worker\cru3_max_2010.csv
Generating URL ...
Please wait ...
Data downloaded to D:\100.Geo_Worker\cru3_min_2010.csv


In [9]:
shp_ee.first().propertyNames().getInfo()


['system:index', 'ELV', 'LAT', 'LON', 'SID', 'x', 'y']

In [13]:
shp_ee.aggregate_array('SID').getInfo()

['AFM00040948',
 'CE000043497',
 'CE000434730',
 'FMC00914482',
 'FMW00040308',
 'FMW00040504',
 'FMW00040505',
 'GQC00914278',
 'GQW00041415',
 'IDM00096179',
 'IN004102500',
 'IN010050700',
 'IN012041000',
 'IN012070800',
 'IN012190100',
 'IN019070100',
 'IN019131301',
 'IN022021900',
 'IN022030600',
 'IN023351400',
 'INM00042071',
 'IOW00070701',
 'IR000040745',
 'IR000040841',
 'IR000040848',
 'IR000408560',
 'JA000047401',
 'JA000047402',
 'JA000047404',
 'JA000047405',
 'JA000047406',
 'JA000047407',
 'JA000047409',
 'JA000047411',
 'JA000047412',
 'JA000047413',
 'JA000047417',
 'JA000047418',
 'JA000047420',
 'JA000047421',
 'JA000047423',
 'JA000047424',
 'JA000047426',
 'JA000047430',
 'JA000047433',
 'JA000047435',
 'JA000047440',
 'JA000047512',
 'JA000047520',
 'JA000047570',
 'JA000047574',
 'JA000047575',
 'JA000047576',
 'JA000047581',
 'JA000047582',
 'JA000047584',
 'JA000047585',
 'JA000047587',
 'JA000047588',
 'JA000047590',
 'JA000047592',
 'JA000047595',
 'JA0000

In [6]:
# year = 2000, 2015
# doy = '-01-01', '-12-31'

# start_date = str(year[0]) + doy[0]
# end_date = str(year[1]) + doy[1]

bound_jx = ee.FeatureCollection('users/yehuigeo/jx_bound')

In [27]:
era5_img = col_era5.select('mean_2m_air_temperature').mean()
scale_tavg = era5_img.projection().nominalScale()
dic = era5_img.reduceRegion(ee.Reducer.mean(),
                      bound_jx.geometry(), 
                      scale=scale_tavg).getInfo()
# reduceRegion(reducer, geometry, scale, crs, crsTransform, bestEffort, maxPixels, tileScale)
# Returns: Dictionary

In [36]:
print(dic)
print(dic.get('mean_2m_air_temperature'))
'{:.2f}'.format(dic.get('mean_2m_air_temperature'))

{'mean_2m_air_temperature': 291.52414219501304}
291.52414219501304


'291.52'

In [7]:
# link_roi = ['users/2431566134liumonarch/QTP',
#            'users/2431566134liumonarch/QTP_Rectangle_CustomProj',
#            'users/2431566134liumonarch/QHandXZ_Province_Proj']
# rois = [ee.FeatureCollection(i) for i in link_roi]
import pandas as pd
roi = bound_jx


list_meto = list()
for year in range(2000, 2015+1):
    image = ee.ImageCollection("ECMWF/ERA5/MONTHLY")\
                .filterDate(str(year), str(year+1))
    tavg_img = image.select('mean_2m_air_temperature').mean()
    prcp_img = image.select('total_precipitation').sum()
    
    scale_tavg = tavg_img.projection().nominalScale()
    scale_prcp = prcp_img.projection().nominalScale()
    
    tavg = tavg_img.reduceRegion(ee.Reducer.mean(), 
                                roi.geometry(), scale=scale_tavg)\
                                .getInfo().get('mean_2m_air_temperature')
    prcp = prcp_img.reduceRegion(ee.Reducer.mean(), 
                                 roi.geometry(), scale=scale_prcp)\
                                .getInfo().get('total_precipitation')
    
    list_meto.append([year, tavg- 273.15, prcp* 1000])

    print(f'{year} 统计结束！！！')
    
df_meto = pd.DataFrame(list_meto, columns=['year','tavg','prcp'])
df_meto.to_csv(r'D:\100.Geo_Worker\ERA5_2000_2019_meteo.csv',float_format='%.2f')



2000 统计结束！！！
2001 统计结束！！！
2002 统计结束！！！
2003 统计结束！！！
2004 统计结束！！！
2005 统计结束！！！
2006 统计结束！！！
2007 统计结束！！！
2008 统计结束！！！
2009 统计结束！！！
2010 统计结束！！！
2011 统计结束！！！
2012 统计结束！！！
2013 统计结束！！！
2014 统计结束！！！
2015 统计结束！！！
